In [1]:
# 基本ライブラリ
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import numpy.random as rd
import gc
import multiprocessing as mp
import os
import sys
import pickle
from collections import defaultdict
from glob import glob
import math
from datetime import datetime as dt
from pathlib import Path
import scipy.stats as st
import re
import shutil
from tqdm import tqdm_notebook as tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # 秒ut(10桁) ⇒ 日付

# グラフ描画系
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc

from matplotlib import animation as ani
from IPython.display import Image

plt.rcParams["patch.force_edgecolor"] = True
#rc('text', usetex=True)
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

#カラム内の文字数。デフォルトは50
pd.set_option("display.max_colwidth", 100)

#行数
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
#
pd.options.display.float_format = '{:,.5f}'.format

%matplotlib inline

In [ ]:
structures = pd.read_csv("../input/structures.csv")

In [26]:
def matrics_rotate_(row, theta):
    # x0 = np.random.random()
    # y1 = 2*math.pi*np.random.random()
    # y2 = 2*math.pi*np.random.random()
    x0 = theta[0]
    y1 = theta[1]
    y2 = theta[2]
    r1 = math.sqrt(1.0-x0)
    r2 = math.sqrt(x0)
    u0 = math.cos(y2)*r2
    u1 = math.sin(y1)*r1
    u2 = math.cos(y1)*r1
    u3 = math.sin(y2)*r2
    coefi = 2.0*u0*u0-1.0
    coefuu = 2.0
    coefe = 2.0*u0
    r = np.zeros(shape=(3, 3))
    r[0, 0] = coefi+coefuu*u1*u1
    r[1, 1] = coefi+coefuu*u2*u2
    r[2, 2] = coefi+coefuu*u3*u3

    r[1, 2] = coefuu*u2*u3-coefe*u1
    r[2, 0] = coefuu*u3*u1-coefe*u2
    r[0, 1] = coefuu*u1*u2-coefe*u3

    r[2, 1] = coefuu*u3*u2+coefe*u1
    r[0, 2] = coefuu*u1*u3+coefe*u2
    r[1, 0] = coefuu*u2*u1+coefe*u3
    return np.dot([row.x, row.y, row.z], r)

rd.seed(SEED)
rotated_structures = []
for i, g in tqdm(structures.groupby("molecule_name")):
    theta = [np.random.random(), 2*np.random.random(), 2*np.random.random()]
    rotated = g.apply(lambda row: matrics_rotate_(row, theta), axis=1, result_type="expand")
    rotated.columns = ["x", "y", "z"]
    rotated["molecule_name"] = i
    rotated["atom_index"] = g.atom_index
    rotated["atom"] = g.atom
    rotated_structures += [rotated]
    # break
    
rotated_structures_df = pd.concat(rotated_structures, axis=0)
rotated_structures_df[structures.columns].to_csv(f"../input/rotated_structures_{SEED}.csv")

In [36]:
assert structures.shape[0] == rotated_structures_df.shape[0]

In [37]:
rotated_structures_df[structures.columns].head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,0.85409,-0.18319,0.64511
1,dsgdb9nsd_000001,1,H,-0.00397,0.00380,-0.00384
2,dsgdb9nsd_000001,2,H,0.93708,0.62072,1.37940
3,dsgdb9nsd_000001,3,H,0.71895,-1.13561,1.16181
4,dsgdb9nsd_000001,4,H,1.76426,-0.22164,0.04306
